In [33]:
import pandas as pd
import numpy as np

In [34]:
# jupyter
train_df = pd.read_csv('./data01/train.csv/train.csv')
test_df = pd.read_csv('./data01/test.csv/test.csv')
submission_df = pd.read_csv('./data01/submission.csv')

# # colab
# from google.colab import drive
# drive.mount('/content/drive')
# train_df = pd.read_csv('/content/drive/MyDrive/kaggle-study/2/data01/train.csv/train.csv')
# test_df = pd.read_csv('/content/drive/MyDrive/kaggle-study/2/data01/test.csv/test.csv')
# submission_df = pd.read_csv('/content/drive/MyDrive/kaggle-study/2/data01/submission.csv')

# 데이터 설명
## 기초 설명

한국의 약 2,000개 상점의 신용카드 거래 내역이 제공됩니다. card_id를 기준으로 샘플링되었으며, amount의 단위는 KRW가 아닙니다. 테스트 파일에서 각 상점의 마지막 매출 발생일 다음 날부터 100일 후까지 매출의 총합을 예측해야 합니다.


예측한 금액만큼 100일간 대출을 진행합니다. 만약 예측한 금액보다 실제 매출이 적게 발생한다면 원금에 손실을 보게 됩니다. 실제 매출이 예측한 금액 이상으로 발생한다면, 연이율 13%의 이자가 발생합니다.


## Data

train.csv - 카드 매출 내역, 2016-08-01부터 2018-07-31까지.


test.csv - train.csv랑 같은 형식. test.csv랑 train.csv의 store_id가 같아도 같은 상점인 것은 아님.

(월에 최소 5일 이상 데이터가 있는 상점만을 문제에 포함.)


submission.csv – store_id와 total_sales 두 개의 열로 구성. store_id는 test.csv의 store_id와 대응.

ore_id가 같아도 같은 상점인 것은 아님.

(월에 최소 5일 이상 데이터가 있는 상점만을 문제에 포함.)


submission.csv – store_id와 total_sales 두 개의 열로 구성. store_id는 test.csv의 store_id와 대응.

## 칼럼

- store_id: 각 파일에서의 상점 고유 번호 (해당기한 내 개업 혹은 폐업한 상점도 있음)
- date: 거래 일자
- time: 거래 시간
- card_id: 카드 번호의 hash 값
- amount: 매출 금액, 음수인 경우 취소 거래
- installments: 할부 개월 수. 일시불은 빈 문자열
- days_of_week: 요일, 월요일이 0, 일요일은 6
- holiday: 1이면 공휴일, 0이면 공휴일 아님

In [35]:
train_df.head()

,store_id,date,time,card_id,amount,installments,days_of_week,holyday
0,0,2016-12-14,18:05:31,d297bba73f,5,NaN,2,0
1,0,2016-12-14,18:05:54,d297bba73f,-5,NaN,2,0
2,0,2016-12-19,12:42:31,0880849c05,144,NaN,0,0
3,0,2016-12-19,12:48:08,8b4f9e0e95,66,NaN,0,0
4,0,2016-12-19,13:31:08,7ad237eed0,24,NaN,0,0


In [36]:
test_df.head()

,store_id,date,time,card_id,amount,installments,days_of_week,holyday
0,0,2016-08-01,00:28:15,bf33518373,125,NaN,0,0
1,0,2016-08-01,01:09:58,7a19a3a92f,90,NaN,0,0
2,0,2016-08-01,01:47:24,6f9fd7e241,150,NaN,0,0
3,0,2016-08-01,17:54:43,8bcf1d61b2,362,NaN,0,0
4,0,2016-08-01,18:48:53,6a722ce674,125,NaN,0,0


In [37]:
submission_df.head()

,store_id,total_sales
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [38]:
train_df['days_of_week'].unique()

array([2, 0, 1, 3, 4, 5, 6], dtype=int64)

In [39]:
train_df.shape

(3362796, 8)

In [40]:
test_df.shape

(473392, 8)

In [41]:
train_df_origin = train_df.copy()

In [42]:
# jupyter -> pandas new version
# train_df.info(show_counts=True)

# null_counts: Deprecated since version 1.2.0 -> Use `show_counts` instead
train_df.info(null_counts=True)

C:\Users\yrc92\.conda\envs\ai_study_py37\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: null_counts is deprecated. Use show_counts instead
  """
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3362796 entries, 0 to 3362795
Data columns (total 8 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   store_id      3362796 non-null  int64  
 1   date          3362796 non-null  object 
 2   time          3362796 non-null  object 
 3   card_id       3362796 non-null  object 
 4   amount        3362796 non-null  int64  
 5   installments  16860 non-null    float64
 6   days_of_week  3362796 non-null  int64  
 7   holyday       3362796 non-null  int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 205.2+ MB


In [43]:
train_df['datetime'] = pd.to_datetime(train_df['date'] + ' ' + train_df['time'])
train_df = train_df.drop(['date', 'time'], axis=1)

In [44]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3362796 entries, 0 to 3362795
Data columns (total 7 columns):
 #   Column        Dtype         
---  ------        -----         
 0   store_id      int64         
 1   card_id       object        
 2   amount        int64         
 3   installments  float64       
 4   days_of_week  int64         
 5   holyday       int64         
 6   datetime      datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 179.6+ MB


In [45]:
train_df = train_df.set_index('datetime')

In [46]:
train_df.head()

,store_id,card_id,amount,installments,days_of_week,holyday
datetime,,,,,,
2016-12-14 18:05:31,0,d297bba73f,5,NaN,2,0
2016-12-14 18:05:54,0,d297bba73f,-5,NaN,2,0
2016-12-19 12:42:31,0,0880849c05,144,NaN,0,0
2016-12-19 12:48:08,0,8b4f9e0e95,66,NaN,0,0
2016-12-19 13:31:08,0,7ad237eed0,24,NaN,0,0


In [47]:
train_df['store_id'].value_counts()

0       68773
753     39318
105     36063
1342    33335
428     33285
        ...  
1605        1
991         1
1263        1
1530        1
632         1
Name: store_id, Length: 1775, dtype: int64

In [48]:
test_df['store_id'].value_counts()

80     32959
77     26163
88     21879
34     20418
153    17651
       ...  
32        60
197       50
130       49
93        49
64        44
Name: store_id, Length: 200, dtype: int64

In [49]:
train_df['installments'][train_df['installments'].isnull()] = 0

C:\Users\yrc92\.conda\envs\ai_study_py37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [50]:
train_df['installments'].value_counts()

0.0     3345936
3.0       10686
2.0        3431
5.0        1220
6.0         647
10.0        252
4.0         209
12.0        206
24.0         96
8.0          32
7.0          28
9.0          25
18.0         11
20.0          6
36.0          5
22.0          4
15.0          2
Name: installments, dtype: int64

데이터가 특정 일 수 미만으로 있으면 제거해야 할 것 같다.

이후 100일의 

In [51]:
train_df.resample('D')['store_id'].value_counts()

datetime    store_id
2016-08-01  105         59
            428         58
            221         56
            597         43
            323         41
                        ..
2018-07-31  1778         1
            1785         1
            1788         1
            1796         1
            1797         1
Name: store_id, Length: 480160, dtype: int64

In [52]:
stores_of_days = train_df.resample('D')['store_id'].unique().values
stores_of_days

700, 1494, 1073,  550,
        181, 1240,   39, 1041,  701,  231,  369, 1224, 1029,    4, 1546,
       1346,   68, 1236,   96, 1429, 1149,  404,  236, 1284,  892,  637,
        126,  985, 1613, 1779,  750, 1047,  886,  966, 1445, 1730,  619,
       1323, 1325, 1245, 1021,  907, 1093, 1709, 1267, 1314,    6,  593,
        801, 1522,   52, 1438, 1209, 1708, 1318,  670,  564,  341,  366,
       1492,  209, 1152, 1380,   32, 1466,  811,  765,  862, 1002, 1405,
        239, 1424,  102, 1297,  920, 1363,  348,  163,  100, 1555,  116,
        931, 1155,  118, 1592,  911, 1731,  175, 1230,  210,  970, 1350,
        358, 1341,  172, 1037,  783, 1553,  812, 1446,  517,  785, 1103,
        158,  462, 1767, 1238,  947,  937, 1453, 1015,  870,  229, 1559,
        469,  329,  442,  249, 1100,  453,  569,  566,  698, 1770, 1413,
        388, 1309,  491, 1540,   59,  930,  739,  914,  215, 1619, 1733,
       1653, 1433,  511,  691, 1246, 1011,  916, 1140,  457,  815, 1593,
        439,   40,  854, 107

In [53]:
from collections import Counter

store_count_by_day = Counter()

for stores_of_day in stores_of_days:
  for store_id in stores_of_day:
    store_count_by_day[store_id] += 1

store_count_by_day

Counter({526: 730,
         942: 730,
         723: 549,
         268: 722,
         1197: 287,
         378: 713,
         188: 670,
         361: 710,
         428: 730,
         1482: 705,
         110: 706,
         1575: 727,
         1343: 697,
         292: 518,
         1107: 688,
         64: 710,
         8: 562,
         1464: 720,
         591: 722,
         1099: 728,
         1062: 730,
         1345: 728,
         394: 725,
         216: 235,
         293: 728,
         24: 661,
         1337: 689,
         1526: 692,
         486: 692,
         1360: 604,
         1406: 721,
         1097: 661,
         782: 713,
         626: 612,
         1361: 644,
         407: 648,
         26: 492,
         1427: 633,
         426: 533,
         1501: 715,
         1306: 287,
         1786: 730,
         1551: 730,
         997: 700,
         957: 586,
         440: 712,
         166: 704,
         555: 730,
         1044: 712,
         1410: 713,
         775: 727,
         109: 

In [54]:
len(store_count_by_day)

1775

In [55]:
store_count_by_day.most_common()

[(526, 730),
 (942, 730),
 (428, 730),
 (1062, 730),
 (1786, 730),
 (1551, 730),
 (555, 730),
 (1677, 730),
 (1716, 730),
 (917, 730),
 (528, 730),
 (221, 730),
 (606, 730),
 (1244, 730),
 (185, 730),
 (1076, 729),
 (356, 729),
 (1487, 729),
 (126, 729),
 (384, 729),
 (1099, 728),
 (1345, 728),
 (293, 728),
 (630, 728),
 (477, 728),
 (1575, 727),
 (775, 727),
 (1566, 727),
 (245, 727),
 (59, 726),
 (215, 726),
 (394, 725),
 (498, 725),
 (309, 725),
 (52, 725),
 (719, 725),
 (1109, 723),
 (1237, 723),
 (1132, 723),
 (1418, 723),
 (998, 723),
 (268, 722),
 (591, 722),
 (462, 722),
 (1616, 722),
 (2, 722),
 (1606, 722),
 (1406, 721),
 (68, 721),
 (1626, 721),
 (1446, 721),
 (22, 721),
 (1023, 721),
 (1323, 721),
 (1464, 720),
 (1392, 720),
 (1495, 720),
 (109, 719),
 (597, 719),
 (946, 718),
 (1771, 717),
 (333, 717),
 (953, 717),
 (13, 717),
 (1161, 716),
 (1670, 716),
 (500, 716),
 (1501, 715),
 (390, 715),
 (1535, 715),
 (51, 714),
 (1768, 714),
 (378, 713),
 (782, 713),
 (1410, 713),
